In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2
import os
from tqdm import tqdm_notebook as tqdm # Jupyter notebook should use this
from numba import njit,prange,cuda
from numba.experimental import jitclass
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from time import time
import logging
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
#from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
from sklearn.decomposition import PCA
#from sklearn.svm import SVC
import os,numpy as np
from skimage.feature import hog
# from skimage import data, exposure
# logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')
from PIL import Image, ImageOps
import glob
from skimage.feature import local_binary_pattern


DATASET_PATH ='15-Scene'

In [2]:
image_list=[];
features_list = [];
label=[]
path=os.getcwd()+'/15-Scene'
print(path)

C:\Users\man14\Downloads\ms/15-Scene


In [3]:
for subfolder in os.listdir(path):
    subpath=path+'/'+subfolder
    print(subpath)
    for filename in os.listdir(subpath): #assuming gif
        #print(subpath+'/'+filename)
        im=Image.open(subpath+'/'+filename)
        im1=im.resize((700,700)) 
        im2=ImageOps.grayscale(im1)
        LBP_image = local_binary_pattern(im2, 16, 1, method='default')
        features_list.append(LBP_image)
        label.append(subfolder)
        image_list.append(im2)# if you need to use image itself not its features

C:\Users\man14\Downloads\ms/15-Scene/00
C:\Users\man14\Downloads\ms/15-Scene/01
C:\Users\man14\Downloads\ms/15-Scene/02
C:\Users\man14\Downloads\ms/15-Scene/03
C:\Users\man14\Downloads\ms/15-Scene/04


In [4]:
X=np.asarray(features_list)
y=np.asarray(label)
target_names =np.unique(y)
n_samples=X.shape[0]
n_features = X.shape[1]
n_classes = target_names.shape[0]

MemoryError: Unable to allocate 4.55 GiB for an array with shape (1245, 700, 700) and data type float64

In [5]:
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)
print('labels: ',target_names)

Total dataset size:
n_samples: 1245
n_features: 700
n_classes: 5
labels:  ['00' '01' '02' '03' '04']


In [6]:
x = X.reshape(n_samples,n_features*n_features)
print(x)

[[2.8672e+04 1.6128e+04 1.6128e+04 ... 1.0000e+00 2.5700e+02 2.5600e+02]
 [6.1441e+04 6.5281e+04 6.1441e+04 ... 5.1100e+02 5.1100e+02 4.9600e+02]
 [6.1441e+04 7.9360e+03 6.5281e+04 ... 2.5700e+02 1.0000e+00 2.5600e+02]
 ...
 [0.0000e+00 1.7920e+03 1.7920e+03 ... 2.6300e+02 7.0000e+00 0.0000e+00]
 [6.1441e+04 7.9360e+03 6.5281e+04 ... 3.1000e+01 3.1000e+01 1.6000e+01]
 [0.0000e+00 3.8400e+03 7.9360e+03 ... 5.1100e+02 5.1100e+02 4.9600e+02]]


In [7]:
print(x.shape)
print(y.shape)


(1245, 490000)
(1245,)


In [8]:
# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

MemoryError: Unable to allocate 3.64 GiB for an array with shape (996, 490000) and data type float64

In [9]:
train_x = X_train/255.
test_x = X_test/255.

In [10]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(train_x,y_train)

RandomForestClassifier()

In [11]:
y_pred = clf.predict(test_x)

In [12]:
print(classification_report(y_test, y_pred, target_names=target_names))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

          00       0.52      0.29      0.37        42
          01       0.44      0.07      0.12        55
          02       0.42      0.65      0.51        54
          03       0.56      0.69      0.62        29
          04       0.62      0.87      0.72        69

    accuracy                           0.53       249
   macro avg       0.51      0.51      0.47       249
weighted avg       0.51      0.53      0.47       249

[[12  3 12  7  8]
 [ 3  4 23  7 18]
 [ 7  1 35  2  9]
 [ 1  0  6 20  2]
 [ 0  1  8  0 60]]


In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5261044176706827